In [123]:
import pickle
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans, DBSCAN
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from collections import defaultdict

In [124]:
with open('museums.pickle', 'rb') as f:
    museums_dict = pickle.load(f)

In [225]:
def preproc_museums(string):
    string = string.strip(' .-').replace('\u200b', '').replace('\xa0', '')
    quotes = list('”»“«"()')
    for quote in quotes:
        string = string.replace(quote, '')
    return string.lower()

In [226]:
list(museums_dict)

['Санкт-Петербург',
 'Москва',
 'Нижний Новгород',
 'Воронеж',
 'Екатеринбург',
 'Пермь',
 'Александров',
 'Краснодар',
 'Чебоксары',
 'Смоленск',
 'Канск',
 'Владивосток',
 'Новосибирск',
 'Челябинск',
 'Ростов-на-Дону',
 'Саратов',
 'Уфа',
 'Бор',
 'Псков',
 'Ижевск',
 'Тула',
 'Унеча',
 'Брянск',
 'Красноярск',
 'Выкса',
 'Обнинск',
 'Сочи',
 'Сокол',
 'Омск',
 'Сургут',
 'Киров',
 'Мурманск',
 'Константиновск',
 'Самара',
 'Выборг',
 'Ханты-Мансийск',
 'Хабаровск',
 'Пущино',
 'Севастополь',
 'Черкесск',
 'Норильск',
 'Тольятти',
 'Ярославль',
 'Казань',
 'Петрозаводск',
 'Знаменск',
 'Калининград',
 'Нижний Тагил',
 'Липецк',
 'Кострома',
 'Томск',
 'Болгар',
 'Белгород',
 'Кемерово',
 'Сим',
 'Гай',
 'Вологда',
 'Грозный',
 'Тамбов',
 'Плес',
 'Ногинск',
 'Электросталь',
 'Коломна',
 'Углич',
 'Первоуральск',
 'Рязань',
 'Гагарин',
 'Шуя',
 'Луга',
 'Тюмень',
 'Анапа',
 'Электроугли',
 'Октябрьский',
 'Владикавказ',
 'Калуга',
 'Златоуст',
 'Донецк',
 'Тверь',
 'Тайга',
 'Лермонт

In [280]:
museums = museums_dict['Москва']
museums = np.array(list(set(list(map(preproc_museums, museums)))))
museums = museums[museums != '']

all_museums = museums


In [281]:
km_clusters = 25
pipeline = Pipeline(steps=[
    ('tfidf', TfidfVectorizer(ngram_range=(1, 2))),
    ('k-means', KMeans(n_clusters=km_clusters))
])

In [282]:
cluster_dict = dict()
mask = np.zeros(museums.shape[0], dtype=bool)

In [283]:
def gen_params():
    eps_list = [i / 10 for i in range(5, 16)]
    n_samples = (6, 5, 4, 4, 3, 3, 2, 2)
    for eps in eps_list:
        for n in n_samples:
            yield eps, n

In [284]:
db_pipeline = Pipeline(steps=[
    ('tfidf', TfidfVectorizer(ngram_range=(1, 2), max_df=0.1)),
    ('dbscan', DBSCAN(eps=0.5, min_samples=2))
])

In [285]:
generator = gen_params()
while len(museums) > len(all_museums) * 2 / 10:
    museums = museums[~mask]
    mask = np.zeros(museums.shape[0], dtype=bool)
    db_pred = db_pipeline.fit_predict(museums)
    db_clusters = len(np.unique(db_pred))
    clustered = db_pred != -1
    mask[clustered] = True
    for j in range(db_clusters - 1): 
        print(j, ' __________________________\n', sep='')
        print(museums[db_pred == j])
        idx = np.argmin([len(elem) for elem in museums[db_pred == j]])
        print(museums[db_pred == j][idx])
        cluster_dict[museums[db_pred == j][idx]] = list(museums[db_pred == j])
    eps, min_samples = next(generator)
    
    db_pipeline.steps[1] = ('dbscan', DBSCAN(eps=eps, min_samples=min_samples))

0 __________________________

['якут галерея' 'якут-галерея']
якут галерея
1 __________________________

["центр современного искусства m'арс" "центр современного искусства м'арс"
 'центр современного искусства м’арс' 'центр современного искусства м`арс']
центр современного искусства m'арс
2 __________________________

['галерея гельмана' 'галерея м.гельмана' 'галерея м & ю гельмана'
 'галерея м. гельмана']
галерея гельмана
3 __________________________

['галерея diehl+gallery one' 'diehl + gallery one'
 'галерея diehl + gallery one' 'diehl+gallery one']
diehl+gallery one
4 __________________________

['школа 109' 'школа №109']
школа 109
5 __________________________

['фонд культуры  екатерина' 'фонд культуры екатерина']
фонд культуры екатерина
6 __________________________

['l-галерея' 'l галерея' 'галерея z & l' 'a-3 галерея' 'галерея л'
 'галерея i.b.s.' 'галерея а' 'галерея' 'галерея а 3' 'галерея a-3'
 'галерея l' 'музей ' 'галерея 9/1' 'галерея 1.0' 'галерея а-3']
музей 
7 ______

['крокус сити' 'крокус-сити']
крокус сити
71 __________________________

['vp- studio' 'vp studio']
vp studio
72 __________________________

['центральный дом архитектора' 'дом архитектора']
дом архитектора
73 __________________________

['0 gallery' '1.0 gallery' 'l gallery' 'z&l gallery']
0 gallery
74 __________________________

['l галерея,  центральный дом художника'
 'l-галерея,  центральный дом художника']
l галерея,  центральный дом художника
75 __________________________

['галерея-витрина: это не здесь' 'галерея-витрина это не здесь']
галерея-витрина это не здесь
76 __________________________

['галерея spider&mouse' 'галерея spider & mouse']
галерея spider&mouse
77 __________________________

['центральный выставочный зал,  манеж'
 'центральный выставочный зал  манеж' 'центральный выставочный зал манеж'
 ' центральный  выставочный зал манеж']
центральный выставочный зал манеж
78 __________________________

['галерея гельман' 'галерея м. и ю. гельман' 'галерея м&ю гельман']
га

0 __________________________

['минералогический музей им. а. е. ферсмана'
 'минералогический музей им. ферсмана ран']
минералогический музей им. ферсмана ран
1 __________________________

['выставочный зал дирекции выставок и аукционов'
 'выставочный зал дирекции выставок и аукционов рфк']
выставочный зал дирекции выставок и аукционов
2 __________________________

['государственный культурный центр-музей им. в. высоцкого / галерея сэм брук'
 'государственный культурный центр-музей в.с. высоцкого/галерея сэм брук']
государственный культурный центр-музей в.с. высоцкого/галерея сэм брук
3 __________________________

['квартирная галерея дмитрия врубеля' 'галерея дмитрия врубеля']
галерея дмитрия врубеля
4 __________________________

['obscuri viri' 'галерея obscuri viri']
obscuri viri
5 __________________________

['граунд песчаная' 'галерея граунд песчаная']
граунд песчаная
6 __________________________

['арт-манеж,  центральный выставочный зал,  манеж'
 'галерея манеж,  арт-манеж,  цен

0 __________________________

['фонд поддержки визуальных искусств елены березкиной эра'
 'фонд поддержки визуальных искусств е. березкиной эра']
фонд поддержки визуальных искусств е. березкиной эра
1 __________________________

['московский дворец молодёжи' 'дворец молодёжи']
дворец молодёжи
2 __________________________

['пересветов пер' 'выставочный зал пересветов пер']
пересветов пер
3 __________________________

['галерея art business consulting'
 'галерея art business consulting,  культурный центр артстрелка']
галерея art business consulting
4 __________________________

['музей современного искусства,  центральный дом художника'
 'государственный центр современного искусства,  центральный дом художника']
музей современного искусства,  центральный дом художника
5 __________________________

['музей вадима сидура в рамках акции ночь в музее'
 'в рамках акции ночь в музее']
в рамках акции ночь в музее
6 __________________________

['центр искусств' 'государственный центр искусств']

0 __________________________

['выставочный зал кузнецкий мост' 'кузнецкий мост'
 'дом художника кузнецкий мост' 'московская галерея,  кузнецкий мост'
 'кузнецкий мост 20']
кузнецкий мост
0 __________________________

['центр дизайна artplay,  галерея риджина'
 'musivum gallery,  центр дизайна artplay' 'центр дизайна'
 'центр дизайна artplay']
центр дизайна
1 __________________________

['третьяковская галерея' 'польша / государственная третьяковская галерея'
 'сша / государственная третьяковская галерея'
 'государственная третьяковская галерея']
третьяковская галерея
0 __________________________

['любительское объединение эрмитаж,  выставочный зал на профсоюзной'
 'творческое объединение эрмитаж' 'любительское объединение эрмитаж'
 'объединение эрмитаж']
объединение эрмитаж
1 __________________________

['цти фабрика' 'и посудой.цти фабрика' 'москва цти фабрика']
цти фабрика
2 __________________________

['выставочный зал на петровских линиях'
 'выставочный зал культурного центра на 

...

0 __________________________

['московский музей архитектуры им. щусева,  крокин галерея'
 'музей архитектуры им']
музей архитектуры им
1 __________________________

['гмии им. а.с. пушкина' 'отдел личных коллекций гмии им. а.с. пушкина']
гмии им. а.с. пушкина
0 __________________________

['центральный дом художника цзх' 'центральный дом художника цдх'
 'сан-франциско / albright knox gallery,  сша / центральный дом художника'
 'сша / центральный дом художника'
 'н. семёновой / центральный дом художника'
 'израиль / центральный дом художника']
центральный дом художника цзх
0 __________________________

['галерея art.ru' 'галерея art+art' 'agency art.ru' 'агенство art.ru']
agency art.ru
1 __________________________

['marat guelman gallery at central house of artists'
 'the central house of artists' 'marat guelman gallery'
 'soros center for contemporary art at central house of artists']
marat guelman gallery
2 __________________________

['галерея глаз / московский центр современного и

2 __________________________

['московский музей современного искусства петровка'
 'московский музей современного искусства гоголевский']
московский музей современного искусства петровка
3 __________________________

["центр современного искусстваm'арс,  музей изобразительных искусств им. а.с.пушкина"
 'музей современного искусства гараж/центр современного искусства  винзавод/государственный музей изобразительных искусств им. а.с. пушкина']
центр современного искусстваm'арс,  музей изобразительных искусств им. а.с.пушкина
4 __________________________

['дворец культуры меридиан' 'дворец культуры мэлз']
дворец культуры мэлз
0 __________________________

['проект новая' 'информационный проект' 'проект невидимые я'
 'проект space' 'авторский проект' 'проект граунд' 'клуб проект о.г.и.'
 'проект в москве' 'проект свободы' 'проект fабrика' 'проект о.г.и.']
проект новая
1 __________________________

['vladey space,  центр современного искусства винзавод'
 'галерея жир,  центр современного ис

галерея зеро,  центральный дом художника
0 __________________________

['агентство арт.ru' 'галерея агентство арт ру']
агентство арт.ru
1 __________________________

['центр фотографии имени братьев люмьер' 'центр фотограии братьев люмьер']
центр фотограии братьев люмьер
2 __________________________

['первая галерея' 'первая галеpея']
первая галерея
3 __________________________

['галерея ravenscourt' 'ravenscourt gallery']
галерея ravenscourt
4 __________________________

['цси винзавод,  цех белого' 'цси винзавод,  коньячный цех']
цси винзавод,  цех белого
5 __________________________

['галерея 1.0,  выставочный зал на солянке,  галерея пропеллер,  галерея садовники'
 'галерея садовники']
галерея садовники
6 __________________________

['фестиваль sretenka design week'
 'в рамках первого фестиваля российского дизайна sretenka design week']
фестиваль sretenka design week
7 __________________________

['московский государственный музей дом бурганова'
 'московский государственный музе


['всероссийский научно-исследовательский и проектно-конструкторский институт металлургического машиностроения'
 'всероссийский научно-исследовательский институт технической эстетики'
 'центр технической эстетики вниитэ'
 'научно-исследовательский институт теоpии и истоpии аpхитектуpы'
 'научно-производственный комплекс научно-исследовательский институт дальней радиосвязи']
центр технической эстетики вниитэ
2 __________________________

['красная площадь' 'лубянская площадь' 'манежная площадь'
 'пушкинская площадь' 'гагаринская площадь,  октябрьская площадь']
красная площадь
0 __________________________

['project space музея современного искусства гараж'
 'при поддержке государственного центра современного искусства и московского музея современного искусства ммси'
 'совместный проект фонда v-a-c и московского музея современного искусства'
 'выставка из коллекций московского музея современного искусства и мультимедиа арт музея'
 'международное портфолио-ревю музей современного искусств

клуб ikra
5 __________________________

['совместно с центром инноваций сколково,  музей современного искусства гараж'
 'аукцион современного искусства vladey' 'школа имени родченко'
 'фестиваль молодого современного искусства россии форма'
 'портрет лица центр современного искусства м’арс организатор – галерея м'
 'государственный центр современного искусства в составе росизо,  фонд культуры екатерина'
 'проект кураторов института современного искусства' 'xl gallery'
 'пространство vladey'
 'галерея xl/московский музей современного искусства/ завод зил и другие'
 "центр м'арс" 'аукцион сотби' 'студия современного искусства артточка'
 'где никому не снятся сны: от священной географии к не-месту. московский музей современного искусства'
 'фестиваль актуальной музыки и искусства nokialab,  школа драматического искусства,  клуб механика'
 'коллекция современного искусства музея-заповедника царицыно' 'ммома '
 'государственный центр музеев и выставок'
 'взаимодействие: взгляд современных х

In [286]:
cluster_dict

{'0 gallery': ['0 gallery', '1.0 gallery', 'l gallery', 'z&l gallery'],
 '11': ['11.12gallery', '11'],
 '11.12 gallery': ['11.12 gallery,  винзавод', '11.12 gallery'],
 '3-й фестиваль видео-арта': ['3-й фестиваль видео-арта',
  'международный фестиваль видео-арта,  государственный музей архитектуры имени а. в. щусева',
  'международный фестиваль видео-арта сейчас&потом’14'],
 '3-я московская биеннале современного искусства': ['3-я московская биеннале современного искусства',
  '1-я московская международная биеннале современного молодого искусства стой! кто идет?музей современного искусства в ермолаевском переулке',
  'в рамках параллельной программы vii московской международной биеннале современного искусства. цти фабрика',
  'в рамках специальной программы iii московской международной биеннале молодого искусства. центр дизайна artplay'],
 'abc': ['галерея - офис abc group', 'abc'],
 'agency art.ru': ['галерея art.ru',
  'галерея art+art',
  'agency art.ru',
  'агенство art.ru'],
 'ars

In [288]:
reverse_dict = dict()
for key in cluster_dict:
    for elem in cluster_dict[key]:
        reverse_dict[elem] = key
for museum in museums:
    reverse_dict[museum] = museum

In [289]:
with open('reverse_museums_dict.pickle', 'wb') as f:
    pickle.dump(reverse_dict, f)